# Welcome to my Coursera Capstone Project!


## My name is Fernando Salazar 

### This is the last part of the IBM Data Science Specialization available on Coursera

In [1]:
import pandas as pd
import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Libraries imported.')
print('Hello Capstone Project Course!')

Libraries imported.
Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto

## Part 1
### Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

In [2]:
# Ensure that we have the required libraries.
!pip install lxml html5lib beautifulsoup4

In [3]:
# URL to Wikipedia: List of postal codes of Canada
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Read the HTML page as Dataframes.
dfs = pd.read_html(url)

print(len(dfs))

3


#### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
# Get the dataframe that contains the table we're interested in.
df = dfs[0]

df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
# Remove rows which Borough is equal to Not Assigned.
df = df[df.Borough != 'Not assigned']

print(df)

    Postal Code           Borough  \
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
..          ...               ...   
160         M8X         Etobicoke   
165         M4Y  Downtown Toronto   
168         M7Y      East Toronto   
169         M8Y         Etobicoke   
178         M8Z         Etobicoke   

                                         Neighbourhood  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
5                     Lawrence Manor, Lawrence Heights  
6          Queen's Park, Ontario Provincial Government  
..                                                 ...  
160      The Kingsway, Montgomery Road, Old Mill North  
165                               Church and Wellesley  
168  Business reply mail Processing Centre

#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [6]:
duplicateDF = df[df.duplicated(['Postal Code'])]

print("Duplicate Rows based on Postal Code are:", duplicateDF, sep='\n')

Duplicate Rows based on Postal Code are:
Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


#### If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [7]:
df.loc[df["Neighbourhood"] == "Not assigned", "Neighbourhood"] = df["Borough"]
# df["Neighbourhood"] = np.where(df["Neighbourhood"] == "Not assigned", df["Borough"], df["Neighbourhood"])
print(df["Neighbourhood"])

2                                              Parkwoods
3                                       Victoria Village
4                              Regent Park, Harbourfront
5                       Lawrence Manor, Lawrence Heights
6            Queen's Park, Ontario Provincial Government
                             ...                        
160        The Kingsway, Montgomery Road, Old Mill North
165                                 Church and Wellesley
168    Business reply mail Processing Centre, South C...
169    Old Mill South, King's Mill Park, Sunnylea, Hu...
178    Mimico NW, The Queensway West, South of Bloor,...
Name: Neighbourhood, Length: 103, dtype: object


D:\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-7-05b090c06653>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df["Neighbourhood"] == "Not assigned", "Neighbourhood"] = df["Borough"]


In [8]:
print(df[df.Neighbourhood == 'Not assigned'])

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


In [9]:
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [10]:
df.shape

(103, 3)

## Part 2
### Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)

In [11]:
geospatial = pd.read_csv("D:\Downloads\Geospatial_Coordinates.csv")

geospatial

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [12]:
# frames = [df, geospatial]
# result = pd.concat(frames)

result = pd.merge(df, geospatial, on='Postal Code')

result

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Part 3
### Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you. 

### Just make sure:

###    1. to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
###    2. to generate maps to visualize your neighborhoods and how they cluster together. 

### Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

In [13]:
resultToronto = result[result['Borough'].str.contains("Toronto")].reset_index(drop=True)

resultToronto

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [14]:
toronto_grouped = resultToronto.groupby('Neighbourhood').mean().reset_index()

toronto_grouped
#resultToronto.groupby('Neighbourhood').count()

,Neighbourhood,Latitude,Longitude
0,Berczy Park,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
2,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
3,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
4,Central Bay Street,43.657952,-79.387383
5,Christie,43.669542,-79.422564
6,Church and Wellesley,43.665860,-79.383160
7,"Commerce Court, Victoria Hotel",43.648198,-79.379817
8,Davisville,43.704324,-79.388790
9,Davisville North,43.712751,-79.390197


In [15]:
toronto_grouped.shape

(39, 3)

In [16]:
# create a new dataframe
# neighborhoods_sorted = pd.DataFrame(columns=columns)
neighborhoods_sorted = toronto_grouped['Neighbourhood']

neighborhoods_sorted 

0                                           Berczy Park
1          Brockton, Parkdale Village, Exhibition Place
2     Business reply mail Processing Centre, South C...
3     CN Tower, King and Spadina, Railway Lands, Har...
4                                    Central Bay Street
5                                              Christie
6                                  Church and Wellesley
7                        Commerce Court, Victoria Hotel
8                                            Davisville
9                                      Davisville North
10                         Dufferin, Dovercourt Village
11               First Canadian Place, Underground city
12      Forest Hill North & West, Forest Hill Road Park
13                             Garden District, Ryerson
14    Harbourfront East, Union Station, Toronto Islands
15                        High Park, The Junction South
16                       India Bazaar, The Beaches West
17            Kensington Market, Chinatown, Gran

In [17]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 0, 1, 1, 3, 1, 1, 4, 4])

In [18]:
# add clustering labels
#neighborhoods_sorted = pd.DataFrame(['Neighbourhood'])
neighborhoods_sorted = toronto_grouped #['Neighbourhood']

neighborhoods_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_sorted

,Cluster Labels,Neighbourhood,Latitude,Longitude
0,1,Berczy Park,43.644771,-79.373306
1,3,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
2,0,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
3,1,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
4,1,Central Bay Street,43.657952,-79.387383
5,3,Christie,43.669542,-79.422564
6,1,Church and Wellesley,43.665860,-79.383160
7,1,"Commerce Court, Victoria Hotel",43.648198,-79.379817
8,4,Davisville,43.704324,-79.388790
9,4,Davisville North,43.712751,-79.390197


In [19]:
#neighborhoods_sorted.drop([Latitude])
neighborhoods_sorted = neighborhoods_sorted.drop(columns=['Latitude','Longitude'])
neighborhoods_sorted

,Cluster Labels,Neighbourhood
0,1,Berczy Park
1,3,"Brockton, Parkdale Village, Exhibition Place"
2,0,"Business reply mail Processing Centre, South C..."
3,1,"CN Tower, King and Spadina, Railway Lands, Har..."
4,1,Central Bay Street
5,3,Christie
6,1,Church and Wellesley
7,1,"Commerce Court, Victoria Hotel"
8,4,Davisville
9,4,Davisville North


In [20]:
toronto_merged = resultToronto

toronto_merged = toronto_merged.join(neighborhoods_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0


In [21]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, CA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, CA are 43.6534817, -79.3839347.


In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters